# 20 Newsgroups & Naive Bayes

This notebook is a modification of 20_Newsgroups_LR example. The motivation for this one is the great dimensionality of feature vectors in the vectorized 20 newsgroups dataset. Maybe even stronger motivation is the relatively low theoretical correlation between individual features, that in fact are term frequencies times inverse document frequency (TF-IDF).

---

I've used a 5-fold cross validation scheme and Parameter Sample for hyperparameter tuning.

---

**Imports**

In [2]:
from __future__ import print_function

%matplotlib inline

import matplotlib.pyplot as plt
plt.style.use("ggplot")

import os
import numpy as np
from sklearn import naive_bayes, preprocessing, cross_validation, datasets, grid_search
import cPickle as cpk
from time import time

/home/alexburlacu/Documents/Projects/MLExperiments/machine-learning-and-a-bit-of-data-science/.venv/local/lib/python2.7/site-packages/sklearn/cross_validation.py:41: DeprecationWarning: This module was deprecated in version 0.18 in favor of the model_selection module into which all the refactored classes and functions are moved. Also note that the interface of the new CV iterators are different from that of this module. This module will be removed in 0.20.
  "This module will be removed in 0.20.", DeprecationWarning)
/home/alexburlacu/Documents/Projects/MLExperiments/machine-learning-and-a-bit-of-data-science/.venv/local/lib/python2.7/site-packages/sklearn/grid_search.py:42: DeprecationWarning: This module was deprecated in version 0.18 in favor of the model_selection module into which all the refactored classes and functions are moved. This module will be removed in 0.20.
  DeprecationWarning)


**Global variables**

In [3]:
NB_FILE_PATH = "../tmp/models/naive_bayes.pkl"
MNB_FILE_PATH = "../tmp/models/multinomial_nb.pkl"
NB_OP_FILE_PATH = "../tmp/models/optimized_naive_bayes.pkl"
MNB_OP_FILE_PATH = "../tmp/models/optimized_multinomial_nb.pkl"

**Loading the dataset**

In [4]:
start = time()
# remove = ('headers', 'footers', 'quotes')
news = datasets.fetch_20newsgroups_vectorized(subset = "all")
print("Data loaded in {0} seconds".format(time() - start))
print("-------------------------------------------------------------------------------")

X = news.data
y = news.target

# _ = [print(topic) for topic in news.target_names] # you may uncoment this to see the topics

print("Number of documents in the dataset: {0}".format(*X.shape))
print("Size of the feature vector of a document: {1}".format(*X.shape))

Data loaded in 1.25855898857 seconds
-------------------------------------------------------------------------------
Number of documents in the dataset: 18846
Size of the feature vector of a document: 130107


**Some utility functions**

In [5]:
def to_str_repr(label):
    return news.target_names[label]

def formated_print(model_name, acc):
    pretty_stat = "Accuracy of the {0} model is {1}.".format(model_name, acc)
    print(pretty_stat)
    
def train_estimator(estimator, features):
    train, _ = next(iter(kfcv))
    estimator.fit(features[train], y[train])
    return estimator

def save_estimator(estimator, filename):
    """
        Note that it is recomended to use '.pkl' or '.model' file extension.
        
        Why? Because I want so.
        ;)
    """
    with open(filename, "wb") as fp:
        cpk.dump(estimator, fp)

def load_estimator(filename):
        with open(filename, "rb") as fp:
            return cpk.load(fp)

def load_or_train(estimator, feature_set, filename):
    if not os.path.exists(filename):
        print("Training...")
        start = time()
        estimator = train_estimator(estimator, feature_set)
        print("Trained in {0} seconds.".format(time() - start))
        print("-------------------------------------------------------------------------------")
        print("Saving...")
        start = time()
        save_estimator(estimator, filename)
        print("Saved in {0} seconds.".format(time() - start))
    else:
        print("Loading...")
        start = time()
        estimator = load_estimator(filename)
        print("Loaded in {0} seconds.".format(time() - start))
        
    return estimator
        
def evaluate_estimator(estimator, features):
    print("Evaluating model's accuracy...")
    start = time()
    score = np.mean(cross_validation.cross_val_score(estimator, features, y, cv = kfcv, n_jobs = 1))
    print("Evaluation done in {0} seconds".format(time() - start))
    print("-------------------------------------------------------------------------------")
    return score

**Feature vector normalization and scaling**

Due to the fact that the vectors are sparse, we must scale them just by the standard variance.

In [6]:
__X = preprocessing.scale(X, with_mean = False)
X_new = preprocessing.normalize(__X)

**5 Fold cross validation**

In order to get more objective results, I have used a 5 fold cross validation scheme, with feature-target pairs shuffling. Because of it, it is now possible to get an averaged result from 5 model evaluations, with training and testing on different portions of the dataset.

In [7]:
kfcv = cross_validation.KFold(len(y), n_folds = 5, shuffle = True)

**Estimators with default parameters**

Almost default, in order to speed up the training process I changed the number of workers from 1 to 3.

In [8]:
gaussian_nb = naive_bayes.GaussianNB()
multinomial_nb = naive_bayes.MultinomialNB()

**Accuracy benchmark for default estimators**

**Gaussian Naive Bayes**

Due to the Memory Error I chose to use `partial_fit()` method, and train and test the model on mini-batches.

In [9]:
# gaussian_nb = load_or_train(gaussian_nb, X, NB_FILE_PATH)
train, test = next(iter(kfcv))

for batch_tr in np.split(train, 3769):
    gaussian_nb.partial_fit(X_new[batch_tr].toarray(), y[batch_tr], classes = range(20))

In [10]:
# mean_acc = evaluate_estimator(gaussian_nb, X)
acc = 0.0
print("Evaluating...")
start = time()
for i, batch in enumerate(np.split(test, 10)):
    current_acc = gaussian_nb.score(X_new[batch].toarray(), y[batch])
    print("Batch Nr.{0} Accuracy {1}".format(i + 1, current_acc))
    acc += current_acc

print("-------------------------------------------------------------------------------")
print("Evaluation done in {0} seconds".format(time() - start))
formated_print("Gaussian Naive Bayes", acc / 10)

Evaluating...
Batch Nr.1 Accuracy 0.811671087533
Batch Nr.2 Accuracy 0.814323607427
Batch Nr.3 Accuracy 0.779840848806
Batch Nr.4 Accuracy 0.790450928382
Batch Nr.5 Accuracy 0.769230769231
Batch Nr.6 Accuracy 0.763925729443
Batch Nr.7 Accuracy 0.718832891247
Batch Nr.8 Accuracy 0.737400530504
Batch Nr.9 Accuracy 0.774535809019
Batch Nr.10 Accuracy 0.708222811671
-------------------------------------------------------------------------------
Evaluation done in 117.328653097 seconds
Accuracy of the Gaussian Naive Bayes model is 0.766843501326.


**Multinomial Naive Bayes**

In [11]:
multinomial_nb = load_or_train(multinomial_nb, X_new, MNB_FILE_PATH)

Training...
Trained in 0.328003168106 seconds.
-------------------------------------------------------------------------------
Saving...
Saved in 3.91957116127 seconds.


In [12]:
mean_acc = evaluate_estimator(multinomial_nb, X_new)

formated_print("Multinomial Naive Bayes", mean_acc)

Evaluating model's accuracy...
Evaluation done in 1.81471395493 seconds
-------------------------------------------------------------------------------
Accuracy of the Multinomial Naive Bayes model is 0.887137973965.


**Hyperparameter selection**

To choose near-optimal hyperparameters I will use Parameter Sample from scikit-learn.
Also it should be mentioned that I will search just for Multinomial Naive Bayes estimator, due to it's superior performance over Gaussian Naive Bayes model.

In [ ]:
multinomial_nb_search_space = {"fit_prior": [True, False],
                               "alpha": [10 ** -x for x in range(5)]
                              }
param_list = grid_search.ParameterSampler(multinomial_nb_search_space, n_iter = 10)

In [ ]:
results = dict()
for i, params in enumerate(param_list):
    nb = naive_bayes.MultinomialNB(**params)
    nb = load_or_train(nb, X_new, "../tmp/tmp{0}.model".format(i))
    acc = evaluate_estimator(nb, X_new)
    results[acc] = params

Loading...


/home/alexburlacu/Documents/Projects/MLExperiments/machine-learning-and-a-bit-of-data-science/.venv/local/lib/python2.7/site-packages/sklearn/base.py:311: UserWarning: Trying to unpickle estimator MultinomialNB from version pre-0.18 when using version 0.19.1. This might lead to breaking code or invalid results. Use at your own risk.
  UserWarning)


Loaded in 7.95780801773 seconds.
Evaluating model's accuracy...


In [ ]:
best_results = list(reversed(sorted(results.keys())))
xs, ys = zip(*map(lambda param: (param["alpha"], param["fit_prior"]), results.values()))

print(best_results)

In [ ]:
cm = plt.cm.get_cmap('Blues')
sc = plt.scatter(xs, ys, c = best_results, vmin = 0.87, vmax = .91, s = 50, cmap = cm)
plt.colorbar(sc)
plt.show()

**Optimized estimators**

After playing around with Parameter Sampler hyperparameter optimizer, I've found the below near optimial configuration.

In [ ]:
multinomial_nb_op = naive_bayes.MultinomialNB(**results[best_results[0]])

**Accuracy benchmark for optimized models**

**Multinomial Naive Bayes**

In [ ]:
multinomial_nb_op = load_or_train(multinomial_nb_op, X_new, MNB_OP_FILE_PATH)

In [ ]:
mean_acc = evaluate_estimator(multinomial_nb_op, X_new)

formated_print("Optimized Multinomial Naive Bayes", mean_acc)

# Conclusion

From all the evaluations done above I can conclude the following 2 things:
- Data preparation, in this case, scaling and normalizing feature vectors is extremely important and leads to major accuracy improvements (Like I didn't knew it...).
- Multinomial Naive Bayes can be trained significantly faster than SGD Classifier, but its accuracy is lower, which means that SGD Classifier is still preferable.